In [358]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [359]:
co2_df = pd.read_csv("./data/GCB2022v27_MtCO2_flat.csv")

In [360]:
# Dropping all rows that have all of the following columsn as NaN
co2_df.dropna(how="all", subset=['Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other'], inplace=True)

In [361]:
# fixng ugly name, fixing the year column, making sure goes from 1960-2021
co2_df.rename({"ISO 3166-1 alpha-3":"COU", "Total":"Total_MTCO2", "Per Capita":"MTCO2_per_cap"}, axis=1, inplace=True)
co2_df.Year = co2_df.Year.astype('int')
co2_df = co2_df.loc[(co2_df['Year'] >= 1960) & (co2_df['Year'] <= 2021)]


In [362]:
# Setting the country and year as the index
co2_df.sort_values(['COU','Year'], inplace=True)
co2_df.set_index(['COU','Year'], inplace=True)


In [363]:
co2_df.drop(np.NaN, level = 0, axis = 0, inplace=True)

In [364]:
# Reading in the country data for GDP per capita
gdp_per_cap_df = pd.read_csv("./data/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_5358417.csv")

In [365]:
# dropping unnecessary columns
gdp_per_cap_df.drop(['Indicator Name', 'Indicator Code', 'Unnamed: 66', "Country Name"], axis = 1, inplace=True)

In [366]:
# renamign columns
gdp_per_cap_df.rename({'Country Code':'COU'}, axis=1, inplace=True)
gdp_per_cap_df.head()

,COU,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25609.955724,26515.678080,26942.307976,28421.386493,28451.273745,29326.708058,30220.594523,31650.760537,24487.863560,29342.100858
1,AFE,162.913034,162.551683,172.002460,199.189238,179.387799,198.230368,209.414665,211.707060,224.239783,...,1759.182395,1730.394686,1719.183721,1538.552268,1443.692371,1628.586788,1564.734340,1512.270553,1363.540741,1549.772730
2,AFG,62.369375,62.443703,60.950364,82.021738,85.511073,105.243196,143.103233,167.165675,134.012768,...,663.141053,651.987862,628.146804,592.476537,520.252064,530.149831,502.056771,500.522664,516.866552,368.754614
3,AFW,106.976475,112.047561,117.730633,122.278715,130.599963,137.186142,142.895375,127.303606,128.365494,...,1953.407033,2149.295219,2243.271464,1876.623483,1645.023767,1585.911930,1731.311792,1749.303317,1683.436391,1757.030626
4,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4962.552072,5101.983876,5059.080441,3100.830685,1709.515534,2283.214233,2487.500996,2142.238757,1603.993477,1953.533757


In [367]:
#melting values to get same format as co2_country_year
gdp_per_cap_df = gdp_per_cap_df.melt(id_vars=['COU'], var_name='Year', value_name='GDP_per_cap')
gdp_per_cap_df

,COU,Year,GDP_per_cap
0,ABW,1960,NaN
1,AFE,1960,162.913034
2,AFG,1960,62.369375
3,AFW,1960,106.976475
4,AGO,1960,NaN
...,...,...,...
16487,XKX,2021,5269.783901
16488,YEM,2021,NaN
16489,ZAF,2021,7055.044776
16490,ZMB,2021,1137.343633


In [368]:
# making sure GDP_per_cap is a float
gdp_per_cap_df.GDP_per_cap = gdp_per_cap_df.GDP_per_cap.astype('float')

In [369]:
# setting index as same
gdp_per_cap_df.sort_values(['COU', 'Year'], inplace=True)
gdp_per_cap_df.Year = gdp_per_cap_df.Year.astype('int')
gdp_per_cap_df.set_index(['COU', 'Year'], inplace=True)


In [370]:
# I know this is alittle weird, but merge was being not very cool, 
#so I did this instead. Joined but had index on CountryCode and Year instead of Country
co2_df = co2_df.join(gdp_per_cap_df, how="inner")
co2_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 12339 entries, ('ABW', 1960) to ('ZWE', 2021)
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country        12339 non-null  object 
 1   Total_MTCO2    12339 non-null  float64
 2   Coal           12276 non-null  float64
 3   Oil            12276 non-null  float64
 4   Gas            12276 non-null  float64
 5   Cement         12178 non-null  float64
 6   Flaring        12276 non-null  float64
 7   Other          1532 non-null   float64
 8   MTCO2_per_cap  12339 non-null  float64
 9   GDP_per_cap    10051 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1012.5+ KB


In [371]:
co2_df.to_csv("./data/processed_data/co2_country_year.csv")

--- Health Stats ---

In [372]:
health_df = pd.read_csv('./data/HEALTH_STAT.csv')
health_df.head()

,VAR,Variable,UNIT,Measure,COU,Country,YEA,Year,Value,Flag Codes,Flags
0,PRHSMBAH,"Bad/very bad health, males aged 15+",PERCALEF,% of population (crude rate),FRA,France,2010,2010,7.8,NaN,NaN
1,PRHSMBAH,"Bad/very bad health, males aged 15+",PERCALEF,% of population (crude rate),FRA,France,2011,2011,7.7,NaN,NaN
2,PRHSMBAH,"Bad/very bad health, males aged 15+",PERCALEF,% of population (crude rate),FRA,France,2012,2012,7.5,NaN,NaN
3,PRHSMBAH,"Bad/very bad health, males aged 15+",PERCALEF,% of population (crude rate),FRA,France,2013,2013,7.9,NaN,NaN
4,PRHSMBAH,"Bad/very bad health, males aged 15+",PERCALEF,% of population (crude rate),FRA,France,2014,2014,7.5,NaN,NaN


In [373]:
#Drop columns we dont need
health_df = health_df.drop(columns=["VAR","UNIT","Measure","YEA","Flag Codes","Flags",'Country'])
health_df

,Variable,COU,Year,Value
0,"Bad/very bad health, males aged 15+",FRA,2010,7.8
1,"Bad/very bad health, males aged 15+",FRA,2011,7.7
2,"Bad/very bad health, males aged 15+",FRA,2012,7.5
3,"Bad/very bad health, males aged 15+",FRA,2013,7.9
4,"Bad/very bad health, males aged 15+",FRA,2014,7.5
...,...,...,...,...
3511,"Bad/very bad health, total aged 15+",HRV,2017,18.4
3512,"Bad/very bad health, total aged 15+",HRV,2018,17.2
3513,"Bad/very bad health, total aged 15+",HRV,2019,16.9
3514,"Bad/very bad health, total aged 15+",HRV,2020,15.0


In [374]:
#We only need values from total population, so drop rows split by demographic
keep_rows = ['Bad/very bad health, total aged 15+',
             'Fair (not good, not bad) health, total aged 15+',
             'Good/very good health, total aged 15+']

split_health_df = health_df[health_df['Variable'].isin(keep_rows)].reset_index(drop=True)
split_health_df

,Variable,COU,Year,Value
0,"Bad/very bad health, total aged 15+",SWE,2010,5.4
1,"Bad/very bad health, total aged 15+",SWE,2011,5.4
2,"Bad/very bad health, total aged 15+",SWE,2012,4.9
3,"Bad/very bad health, total aged 15+",SWE,2013,4.6
4,"Bad/very bad health, total aged 15+",SWE,2014,4.6
...,...,...,...,...
1167,"Bad/very bad health, total aged 15+",HRV,2016,18.8
1168,"Bad/very bad health, total aged 15+",HRV,2017,18.4
1169,"Bad/very bad health, total aged 15+",HRV,2018,17.2
1170,"Bad/very bad health, total aged 15+",HRV,2019,16.9


In [375]:
#Rename columns
split_health_df.rename({'Variable':'PrecievedHealth', 'Value':'PercentOfPopulation'}, inplace=True, axis=1)
split_health_df

,PrecievedHealth,COU,Year,PercentOfPopulation
0,"Bad/very bad health, total aged 15+",SWE,2010,5.4
1,"Bad/very bad health, total aged 15+",SWE,2011,5.4
2,"Bad/very bad health, total aged 15+",SWE,2012,4.9
3,"Bad/very bad health, total aged 15+",SWE,2013,4.6
4,"Bad/very bad health, total aged 15+",SWE,2014,4.6
...,...,...,...,...
1167,"Bad/very bad health, total aged 15+",HRV,2016,18.8
1168,"Bad/very bad health, total aged 15+",HRV,2017,18.4
1169,"Bad/very bad health, total aged 15+",HRV,2018,17.2
1170,"Bad/very bad health, total aged 15+",HRV,2019,16.9


In [376]:
#Recode Variable column to be more readable
code = {
    'Bad/very bad health, total aged 15+' : 'Bad',
    'Fair (not good, not bad) health, total aged 15+' : 'Fair',
    'Good/very good health, total aged 15+' : 'good'
}

split_health_df['PrecievedHealth'].mask(split_health_df['PrecievedHealth'] == 'Bad/very bad health, total aged 15+', 'Bad', inplace=True)
split_health_df['PrecievedHealth'].mask(split_health_df['PrecievedHealth'] == 'Fair (not good, not bad) health, total aged 15+', 'Fair', inplace=True)
split_health_df['PrecievedHealth'].mask(split_health_df['PrecievedHealth'] == 'Good/very good health, total aged 15+', 'Good', inplace=True)
split_health_df

,PrecievedHealth,COU,Year,PercentOfPopulation
0,Bad,SWE,2010,5.4
1,Bad,SWE,2011,5.4
2,Bad,SWE,2012,4.9
3,Bad,SWE,2013,4.6
4,Bad,SWE,2014,4.6
...,...,...,...,...
1167,Bad,HRV,2016,18.8
1168,Bad,HRV,2017,18.4
1169,Bad,HRV,2018,17.2
1170,Bad,HRV,2019,16.9


In [377]:
# Getting ready for join
split_health_df.sort_values(['COU','Year'], inplace=True)
split_health_df.Year = split_health_df.Year.astype('int')
split_health_df.set_index(['COU','Year'], inplace=True)
split_health_df

PrecievedHealth  PercentOfPopulation
COU Year                                     
AUS 2011             Bad                  4.0
    2011            Fair                 10.7
    2011            Good                 85.4
    2014             Bad                  4.4
    2014            Fair                 10.4
...                  ...                  ...
USA 2018            Good                 87.9
    2018             Bad                  2.6
    2018            Fair                  9.0
    2019             Bad                  3.3
    2019            Fair                 10.9

[1172 rows x 2 columns]

In [378]:
# Limmiting CO2 to the years available in percievedhealth
co2_df = co2_df.loc[(slice(None),slice(2011, 2019)), :]

In [379]:
co2_df = co2_df.join(split_health_df, how='left')

In [380]:
co2_df.to_csv("./data/processed_data/co2_country_year.csv")